In [51]:
from Preprocessing import * 
import re

In [52]:
#1-Read Datasets
training_dataset = read_training_dataset()
read_dev_dataset()

Read training set successfully
Read validation set successfully


['( 27 ) قَوْلُهُ : وَلَا تُكْرَهُ ضِيَافَتُهُ .',
 '( الْفَرْقُ الثَّالِثُ وَالثَّلَاثُونَ بَيْنَ قَاعِدَةِ تَقَدُّمِ الْحُكْمِ عَلَى سَبَبِهِ دُونَ شَرْطِهِ أَوْ شَرْطِهِ دُونَ سَبَبِهِ وَبَيْنَ قَاعِدَةِ تَقَدُّمِهِ عَلَى السَّبَبِ وَالشَّرْطِ جَمِيعًا ) وَتَحْرِيرُهُ أَنَّ الْحُكْمَ إنْ كَانَ لَهُ سَبَبٌ بِغَيْرِ شَرْطٍ فَتَقَدَّمَ عَلَيْهِ لَا يُعْتَبَرُ أَوْ كَانَ لَهُ سَبَبَانِ أَوْ أَسْبَابٌ فَتَقَدَّمَ عَلَى جَمِيعِهَا لَمْ يُعْتَبَرْ أَوْ عَلَى بَعْضِهَا دُونَ بَعْضٍ اُعْتُبِرَ بِنَاءً عَلَى سَبَبِ الْخَاصِّ وَلَا يَضُرُّ فِقْدَانُ بَقِيَّةِ الْأَسْبَابِ فَإِنَّ شَأْنَ السَّبَبِ أَنْ يَسْتَقِلَّ بِثُبُوتِ مُسَبِّبِهِ دُونَ غَيْرِهِ مِنْ الْأَسْبَابِ مِثَالُ الْأَوَّلِ الزَّوَالُ سَبَبُ وُجُوبِ الظُّهْرِ فَإِذَا صَلَّيْت قَبْلَ الزَّوَالِ لَمْ تُعْتَبَرْ ظُهْرًا وَمِثَالُ الثَّانِي الْجَلْدُ لَهُ ثَلَاثَةُ أَسْبَابٍ الزِّنَى وَالْقَذْفُ وَالشُّرْبُ فَمَنْ جُلِدَ قَبْلَ مُلَابَسَةِ شَيْءٍ مِنْ هَذِهِ الثَّلَاثَةِ لَمْ يُعْتَبَرْ ذَلِكَ حَدًّا وَلَا زَجْرًا فَهَذَانِ قِسْمَانِ م

In [53]:
# 2-Data cleaning
# Define a regular expression pattern
# This pattern keeps Arabic letters, diacritics, and whitespaces and endlines
# pattern = re.compile(r"[^\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF\s\n]")
pattern = re.compile(r'[^\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF\s,.؟،;:!?\-\'"]')



# Replace unmatched characters with an empty string

cleaned_corpus = [re.sub(pattern, "", t) for t in training_dataset]
cleaned_corpus = [re.sub("\s\s+", " ", c) for c in cleaned_corpus]
# re.sub("\s\s+" , " ", s)


print(len(cleaned_corpus))
# write the clean corpus to file
with open("cleaned_corpus.txt", "w", encoding="utf-8") as f:
    for l in cleaned_corpus:
        f.write(l + "\n")

50000


In [54]:
def extract_arabic_letters(input_word):
    # Define a regular expression pattern for standard Arabic letters
    # pattern = re.compile(r"[\u0600-\u06FF]+") #NOTE returns all letter and dialects
    pattern = re.compile(r"[\u0621-\u064A\s]+")

    # Find all matches and join them
    # This will exclude non-letter characters and diacritics
    result = "".join(pattern.findall(input_word))

    return result


# data_with_labels = []
data = []
labels = []
# Example usage
for c in cleaned_corpus:
    sentences = re.split(r"[,.؟،;:!?']+", c)  # split on all punctuation
    labels += sentences

    without_dialects = [
        extract_arabic_letters(s) for s in sentences
    ]  # get the letters without dialects
    data += without_dialects

# remove empty lines
# data = [d for d in data if not re.match("[\s\n]+", d)]
# labels = [l for l in labels if not re.match("[\s\n]+", l)]
# write the clean corpus to file
with open("training_data.txt", "w", encoding="utf-8") as f:
    for l in data:
        f.write(str(l) + "\n")

with open("training_labels.txt", "w", encoding="utf-8") as f:
    for l in labels:
        f.write(str(l) + "\n")

In [55]:
print(len(data))
print(len(labels))

212012
212012


In [56]:
# 3-Tokenization
# print(tokenize(train[0]))

NameError: name 'train' is not defined